In [ ]:
#!pip install missingno

In [ ]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import missingno as msno
from scipy.stats import chi2_contingency
from tqdm import tqdm
import scipy.stats as st
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, log_loss, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_curve, auc, log_loss
from itertools import product
from collections import Counter
from sklearn.base import clone
from sklearn.metrics import accuracy_score, hamming_loss
from sklearn.utils import check_X_y, check_array
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_val_predict
import shap

## PCC

In [ ]:
full_df = pd.read_csv('OutputData/mental_health_full.csv')

# ------------------ 1. Data Preparation ------------------ #
df_pcc = full_df[
    full_df['q26'].isin([0, 1]) &
    full_df['q27'].isin([0, 1]) &
    full_df['q84_binary'].isin([0, 1])
].copy()

X_pcc = df_pcc.drop(columns=['q26', 'q27', 'q84', 'q84_binary', 'state', 'sitename', 'q28'])
Y_pcc = df_pcc[['q26', 'q27', 'q84_binary']]

X_pcc = X_pcc.select_dtypes(include=[float, int]).dropna()
Y_pcc = Y_pcc.loc[X_pcc.index]

X_train_pcc, X_test_pcc, Y_train_pcc, Y_test_pcc = train_test_split(
    X_pcc, Y_pcc, test_size=0.3, random_state=42, stratify=Y_pcc['q26']
)

scaler = StandardScaler()
X_train_pcc[['bmipct']] = scaler.fit_transform(X_train_pcc[['bmipct']])
X_test_pcc[['bmipct']] = scaler.transform(X_test_pcc[['bmipct']])

# ------------------ 2. PCC Class ------------------ #
class ProbabilisticClassifierChain:
    def __init__(self, baselearner):
        self.baselearner = baselearner
        self.r_ = None
        self.fitted_ = None
        self.patterns_ = None

    def fit(self, x, y):
        x, y = check_X_y(x, y, multi_output=True)
        _, r = y.shape
        self.r_ = r
        self.fitted_ = []
        self.patterns_ = [list(p) for p in product(*(r * [range(2)]))]
        for i in range(r):
            _x = np.column_stack([x, y[:, :i]]) if i > 0 else x
            _y = y[:, i]
            model = clone(self.baselearner, safe=False)
            model.fit(_x, _y)
            self.fitted_.append(model)
        return self

    def predict_proba_of(self, x, y):
        _x = x.copy()
        if len(y.shape) == 1:
            y = y.reshape(1, self.r_)
        idx = np.arange(len(x))
        res = self.fitted_[0].predict_proba(_x)[idx, y[:, 0]]
        for i in range(1, self.r_):
            _x = np.column_stack([_x, np.ones(len(x)) * y[:, i - 1]])
            res *= self.fitted_[i].predict_proba(_x)[idx, y[:, i]]
        return res

    def predict(self, x):
        x = check_array(x)
        pattern_matrix = np.array([self.predict_proba_of(x, np.array(p)) for p in self.patterns_]).T
        max_indices = np.argmax(pattern_matrix, axis=1)
        return np.array([self.patterns_[i] for i in max_indices])

# ------------------ 3. Train ------------------ #
pcc_model = ProbabilisticClassifierChain(LogisticRegression(solver='liblinear', max_iter=1000))
pcc_model.fit(X_train_pcc.to_numpy(), Y_train_pcc.to_numpy())

# ------------------ 4. Extract Top Features ------------------ #

top_features_per_target = []

# Loop through each fitted model and collect top features
for i, clf in enumerate(pcc_model.fitted_):
    coefs = clf.coef_.ravel()
    valid_indices = np.arange(len(coefs))
    abs_coefs = np.abs(coefs)

    # Sort and select top indices safely
    top_indices = valid_indices[np.argsort(abs_coefs)[::-1][:min(10, len(coefs))]]

    # Get column names for top features
    top_feats = [X_train_pcc.columns[i] for i in top_indices if i < X_train_pcc.shape[1]]
    top_features_per_target.extend(top_feats)

# Count frequency of each feature
top_counts = Counter(top_features_per_target)

# Show 10 most common features across all 3 sub-models
print("\nTop 10 Most Frequent Features Across PCC Sub-models:")
for feat, count in top_counts.most_common(10):
    label = feature_labels.get(feat, feat)
    print(f"- {label} (appeared in {count}/3 models)")


# ------------------ 5. Evaluation Metrics ------------------ #

# Predict
Y_pred = pcc_model.predict(X_test_pcc.to_numpy())
Y_true = Y_test_pcc.to_numpy()

# (a) Accuracy per target
print("\n(a) Accuracy per Target:")
for i, col in enumerate(Y_test_pcc.columns):
    acc = accuracy_score(Y_true[:, i], Y_pred[:, i])
    print(f"Accuracy for {col}: {acc:.4f}")

# (b) Hamming Loss (average error per variable)
print("\n(b) Hamming Loss (average error per variable):")
print(f"{hamming_loss(Y_true, Y_pred):.4f}")

# (c) Exact Match (0/1 Loss) – all targets predicted correctly
print("\n(c) Exact Match (0/1 Loss):")
exact_match = np.mean(np.all(Y_true == Y_pred, axis=1))
print(f"{exact_match:.4f}")

# (d) Log Loss (multi-output, joint probability of full target vector)
print("\n(d) Log Loss (multi-output):")
probs = np.column_stack([
    pcc_model.predict_proba_of(X_test_pcc.to_numpy(), np.array(p))
    for p in pcc_model.patterns_
])

true_patterns = [tuple(row) for row in Y_true]
pattern_map = {tuple(p): i for i, p in enumerate(pcc_model.patterns_)}
true_indices = [pattern_map[tuple(p)] for p in true_patterns]
true_probs = probs[np.arange(len(probs)), true_indices]
logloss = -np.mean(np.log(true_probs + 1e-10))  # Add epsilon for stability
print(f"{logloss:.4f}")


print("\n(e) False Positive Rate (FPR) and False Negative Rate (FNR) per Target:")
for i, col in enumerate(Y_test_pcc.columns):
    cm = confusion_matrix(Y_true[:, i], Y_pred[:, i])
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0
    print(f"{col}: FPR = {fpr:.4f}, FNR = {fnr:.4f}")



Top 10 Most Frequent Features Across PCC Sub-models:
- gdp 2023 (appeared in 3/3 models)
- Mean household income (appeared in 3/3 models)
- Unemployment Rate(Percent) (appeared in 3/3 models)
- Sexual and gender identity (appeared in 3/3 models)
- Saw physical violence in neighborhood (appeared in 3/3 models)
- Sex (appeared in 2/3 models)
- Ever had sexual intercourse (appeared in 2/3 models)
- Electronic bullying (past 12 months) (appeared in 2/3 models)
- Bullying at school (past 12 months) (appeared in 2/3 models)
- BMI percentile (appeared in 2/3 models)

(a) Accuracy per Target:
Accuracy for q26: 0.5726
Accuracy for q27: 0.8299
Accuracy for q84_binary: 0.7241

(b) Hamming Loss (average error per variable):
0.2911

(c) Exact Match (0/1 Loss):
0.5062

(d) Log Loss (multi-output):
1.7560

(e) False Positive Rate (FPR) and False Negative Rate (FNR) per Target:
q26: FPR = 0.0000, FNR = 1.0000
q27: FPR = 0.0000, FNR = 1.0000
q84_binary: FPR = 0.0000, FNR = 1.0000
